In [1]:
import zipfile
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
from datetime import datetime as dt, timedelta
from IPython.display import display, HTML
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import functions as f

In [2]:
filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'

In [3]:
%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)
#for chunk in pd.read_csv(filename,chunksize=chunksize,sep='|',low_memory=False):
    #DF = pd.concat([DF,chunk])

CPU times: user 49.6 ms, sys: 108 ms, total: 157 ms
Wall time: 216 ms


In [4]:
%%time
df = chunks.get_chunk(1000000)

CPU times: user 14.7 s, sys: 5.25 s, total: 19.9 s
Wall time: 23.1 s


In [63]:
df.head()

,BIRTH_DATE,CHANNEL,CLOSEST_STORE1_DIST,DAY_OF_THE_WK,FIRST_CATALOG_ORDER_DATE,FIRST_ORDER_CHANNEL,FIRST_ORDER_DATE,FIRST_RETAIL_ORDER_DATE,FIRST_WEB_ORDER_DATE,FISCAL_DAY_OF_THE_YR,FISCAL_MO,FISCAL_QTR,FISCAL_WK,FISCAL_YR,ILINK,IS_CALLABLE_IND,IS_EMAILABLE_IND,IS_JJCH_IND,JJCH_CLOSE_DATE,JJCH_OPEN_DATE,MASTER_CHANNEL,ORDER_DATE,ORDER_DETAIL_KEY,ORDER_KEY,ORIGINAL_RETAIL_PRICE_AMT,PRICE_CD,ORDER_SHIPPING_AMT,PAY_TYPE_CD,SHIPPED_COST_AMT,SHIPPED_GROSS_AMT,SHIPPED_QTY,SHIPPED_SOLD_AMT,SKU_CD,STATE,STYLE_NB,ZIP,FABRIC_CATEGORY_DESC,BODY_NB,DESCRIPTION,EXTENDED_SIZE_DESC,COLOR_FAMILY_DESC,END_USE_DESC,STYLETYPEDESC,CLASS_NAME,DEPARTMENT_NAME,Source,Days_Lapsed,Days_Lapsed_Known,Days_Lapsed_LB,Recency_Month,Recency_Quarter,Recency_Year,PRIOR_ORDER_DATE,margin,discount,markdown,discount_plus
0,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484654,7094417803,79.0,SP,0.0,MC,16.1729,49.99,1,35.99,07569114XL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8171,14.0,29.01,43.01
1,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936659,7094417803,79.0,SP,0.0,MC,16.1900,59.99,1,35.99,07569114XXL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8000,24.0,19.01,43.01
2,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936661,7094417803,79.0,SP,0.0,MC,18.6200,39.99,1,23.99,0808010OE1SZ,MA,80801,1473,Wool/Wool Bl,80801,Terrace Scroll Scarf,No Ext Size,Red,Core,Accessories,Other Scarves,Fashion Scarves,3,4247,0,1066.0,React,React,React,6/10/2005,5.3700,16.0,39.01,55.01
3,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484652,7094417803,89.0,SP,0.0,MC,NaN,0.00,0,0.00,075781LLBL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,NaN,0.0,89.00,89.00
4,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936660,7094417803,89.0,SP,0.0,MC,20.1079,69.99,1,41.99,075781LLBXL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,21.8821,28.0,19.01,47.01


In [ ]:
#only selecting orders that happened between may 2017-july 2017 for sample size

In [5]:
%%time
DF_SAMPLE = df.loc[df.IS_EMAILABLE_IND=='Y']
DF = DF_SAMPLE.copy()
DF['ORDER_DATE'] = DF.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
df1 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
df2 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,6,1).date())]
df3 = DF.loc[(DF.ORDER_DATE >= dt(2017,6,1).date()) & 
              (DF.ORDER_DATE < dt(2017,7,1).date())]
df4 = DF.loc[(DF.ORDER_DATE >= dt(2017,7,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
#df1 = df1.loc[(df1.ORDER_DATE.dt.month >= 5) & (df1.ORDER_DATE.dt.month < 8)]
#df1 = df1[~df1.BIRTH_DATE.isnull()]

CPU times: user 13.5 s, sys: 743 ms, total: 14.2 s
Wall time: 14.6 s


In [285]:
#print 'unique customer ids:', len(df1.ILINK.unique())
#print 'Number emailable:', df1.shape[0]

unique customer ids: 5597
Number emailable: 40180


In [286]:
df1['BIRTH_DATE'] = df1.BIRTH_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_CATALOG_DATE'] = df1.FIRST_CATALOG_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
df1['FIRST_ORDER_DATE'] = df1.FIRST_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_RETAIL_ORDER_DATE'] = df1.FIRST_RETAIL_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_WEB_ORDER_DATE'] = df1.FIRST_WEB_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_CLOSE_DATE'] = df1.JJCH_CLOSE_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_OPEN_DATE'] = df1.JJCH_OPEN_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['PRIOR_ORDER_DATE'] = df1.PRIOR_ORDER_DATE.apply(lambda x: dt.strptime(x,'%m/%d/%Y').date())

# Definitions

In [6]:
def normalize(l):
    s = sum(l)
    if s == 0:
        return [0]*8
    else:
        return [i/s for i in l]

##standardize variables
def standardization_f(x):
    x_bar = np.mean(x)
    s = np.std(x)
    if s == 0:
        return 0
    else:
        x_z = (x-x_bar)/s
    return(x_z)

In [289]:
#make empty dataframe to add onto with customer features
columns = ['Age',
           'WovenShirts_N','KnitTops_N','Pants_N','Dresses_N',
           'WovenShirts_AvgN_Order','KnitTops_AvgN_Order','Pants_AvgN_Order','Dresses_AvgN_Order',
           'WovenShirts_Total','KnitTops_Total','Pants_Total','Dresses_Total',
           'WovenShirts_Amt_Order','KnitTops_Amt_Order','Pants_Amt_Order','Dresses_Amt_Order',
           'WovenShirts_AvgAmt','KnitTops_AvgAmt','Pants_AvgAmt','Dresses_AvgAmt']


# Exploration


In [410]:
reload(f)

<module 'functions' from 'functions.py'>

In [ ]:
#Get Pref/rating over course of May 1 2017 - July 31 2017

In [133]:
ilinks = df1.ILINK.unique().tolist()
users = ilinks[:1000] #sample 

In [7]:
mergedf = df2.merge(df3,on='ILINK')
ilinks = mergedf.ILINK.unique().tolist()
users = ilinks[:1000]#sample
print len(ilinks)

1286


In [8]:
%%time
cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
#cols2 = ['Ilink','N_ws','S_ws','s_ws','si_ws','R_ws',
#        'N_kt','S_kt','s_kt','si_kt','R_kt',
#        'N_d','S_d','s_d','si_d','R_d',
#        'N_p','S_p','s_p','si_p','R_p']
#cols = ['Ilink','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
#       'S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
#        'S_d','s_d','si_d','D_d','d_d','di_d','R_d',
#        'S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = pd.DataFrame([],columns=cols)

#need to create rows that describe a customers freq and sales of items in specific departments
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
K = 4 #Number of Depts/topics
UserSigma = {}
for ilink in users:
    dfCust = df1.loc[(df1.ILINK==ilink)]
    N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
    fts = []
    #fts2 = []
    UserSigma[ilink] = {}
    for dept in depts:
        N = f.getTotalFreq(dfCust,dept)
        n = f.getAvgFreqOrder(dfCust,dept)
        O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
        o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
        oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
        alpha = [N,n,S,s,si,D,d,di]
        #alpha = [S,s,si,D,d,di]
        #alpha = [N,S,s,si]
        alpha = [0 if math.isnan(x) else x for x in alpha]
        alpha1 = np.array(normalize(alpha))
        sigma = (N + alpha1)/(N_u + K*alpha1)
        UserSigma[ilink][dept] = sigma
        pref = np.dot(sigma,alpha1).round(2)
        #print ilink, alpha1
        #print sigma
        #print ''
        fts.append(alpha+[pref])
        #fts2.append(alpha)
    
    """u,s,vt = np.linalg.svd(fts2)
    for i,row in enumerate(fts2):
        fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
    row = [i for sub in fts for i in sub]
    row.insert(0,ilink)
    temp = pd.DataFrame([tuple(row)],columns=cols)
    table = pd.concat([table,temp])

CPU times: user 2min 37s, sys: 2.2 s, total: 2min 39s
Wall time: 2min 51s


In [9]:
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = table.loc[~(table[cols]==0).all(axis=1)]

In [10]:
table.reset_index(drop=True,inplace=True)

In [62]:
table.head()

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
0,800,3,2.0,139.12,69.56,46.37,77.88,38.94,25.96,0.67,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.0,0.00,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0
1,825,1,0.0,34.19,8.55,34.19,25.80,6.45,25.80,0.08,2,0.0,46.20,11.55,23.1,32.79,8.20,16.39,0.15,0,0.0,0.0,0.00,0,0.0,0.0,0,0,4,1.0,138.87,34.72,34.72,100.10,25.02,25.02,0.28
2,845,8,3.0,292.99,97.66,36.62,162.00,54.00,20.25,0.22,4,1.0,118.75,39.58,29.69,69.24,23.08,17.31,0.11,0,0.0,0.0,0.00,0,0.0,0.0,0,0,14,5.0,514.21,171.40,36.73,265.79,88.60,18.99,0.37
3,872,1,0.0,52.53,6.57,52.53,26.47,3.31,26.47,0.05,13,2.0,437.99,54.75,33.69,174.95,21.87,13.46,0.52,1,0.0,119.0,14.88,119,0.0,0.0,0,0.06,4,0.0,191.28,23.91,47.82,95.71,11.96,23.93,0.17
4,896,8,1.0,343.59,28.63,42.95,140.41,11.70,17.55,0.15,34,3.0,940.48,78.37,27.66,359.46,29.96,10.57,0.6,0,0.0,0.0,0.00,0,0.0,0.0,0,0,6,0.0,293.65,24.47,48.94,111.35,9.28,18.56,0.11


# Investigating If R has any predictivability

In [12]:
def getTableRating(ilinks,dataframe):
    cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    tab = pd.DataFrame([],columns=cols)
    #need to create rows that describe a customers freq and sales of items in specific departments
    depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
    K = 4 #Number of Depts/topics
    for ilink in ilinks:
        dfCust = dataframe.loc[(dataframe.ILINK==ilink)]
        if ~dfCust.empty:
            N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
            #print ilink,N_u
            fts = []
            for dept in depts:
                N = f.getTotalFreq(dfCust,dept)
                n = f.getAvgFreqOrder(dfCust,dept)
                O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
                o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
                oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
                alpha = [N,n,S,s,si,D,d,di]
                #alpha = [S,s,si,D,d,di]
                #alpha = [N,S,s,si]
                alpha = [0 if math.isnan(x) else x for x in alpha]
                alpha1 = np.array(normalize(alpha))
                sigma = UserSigma[ilink][dept]
                #sigma = (N + alpha1)/(N_u + K*alpha1)
                pref = np.dot(sigma,alpha1).round(2)
                #print ilink,alpha
                #print sigma
                #print ''
                fts.append(alpha+[pref])
                #fts2.append(alpha)

            """u,s,vt = np.linalg.svd(fts2)
            for i,row in enumerate(fts2):
                fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
            row = [i for sub in fts for i in sub]
            row.insert(0,ilink)
            temp = pd.DataFrame([tuple(row)],columns=cols)
            tab = pd.concat([tab,temp])
    return tab

## 1. First we get what Users pref for May-June

In [13]:
def getHigherPref(row):
    ws = (row.R_ws,row.N_ws,row.S_ws)
    kt = (row.R_kt,row.N_kt,row.S_kt)
    d = (row.R_d,row.N_d,row.S_d)
    p = (row.R_p,row.N_p,row.S_p)
    R = {'Woven Shirts': ws, 'Knit Tops': kt, 'Dresses': d, 'Pants': p}
    maxNum = 0.0
    maxKey = []
    #get those with highest rating first
    for key,value in R.iteritems():
        if value[0] > maxNum:
            maxNum = value[0]
            maxKey.append(key)
        elif value[0] == maxNum and value[0] != 0:
            maxKey.append(key)
    
    #if no purchases were made
    if len(maxKey) == 0:
        return 0
    #if rating are equal, then look at freq purchased for pref
    i = 1
    while len(maxKey) != 1:
        RTop = {x:R[x] for x in maxKey}
        n = 0
        maxKey = []
        for key,value in RTop.iteritems():
            #print value[i]
            if value[i] > n:
                n = value[i]
                if len(maxKey)==0:
                    maxKey.append(key)
                else:
                    maxKey.pop()
                    maxKey.append(key)
            elif value[i] == n and value[i] !=0:
                maxKey.append(key)
        #print maxKey,n
        if len(maxKey) > 1:
            i += 1
            if i > 2:
                return maxKey
    return maxKey[0]

In [14]:
%%time
Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N=0
for i,row in table.iterrows():
    if N%100 == 0:
        print N
    N+=1
    #print row.Ilink
    key = getHigherPref(row)
    if type(key) is list:
        for x in key:
            Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        Users[key].append(row.Ilink)

0
100
200
300
400
500
600
700
800
900
CPU times: user 446 ms, sys: 16 ms, total: 462 ms
Wall time: 531 ms


In [26]:
#table.Ilink.tolist()
#first exmaple will be pants
dept = 'Pants'
ilink_ex = Users[dept]
len(Users[dept])

272

## 2. Look at these Users in May-June

In [17]:
table2 = getTableRating(ilink_ex,df2)
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table2 = table2.loc[~(table2[cols]==0).all(axis=1)]
table2.reset_index(drop=True,inplace=True)

In [18]:
#table2.sort_values('R_p',ascending=False)
table2.head()

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
0,825,1,0.0,34.19,17.10,34.19,25.80,12.90,25.80,0.08,1,0.0,22.80,11.40,22.80,17.19,8.60,17.19,0.15,0,0.0,0.0,0.0,0,0.0,0.0,0,0,3,2.0,85.47,42.74,28.49,64.50,32.25,21.50,0.28
1,845,7,4.0,243.83,121.92,34.83,142.16,71.08,20.31,0.22,4,2.0,118.75,59.38,29.69,69.24,34.62,17.31,0.11,0,0.0,0.0,0.0,0,0.0,0.0,0,0,10,5.0,324.68,162.34,32.47,189.32,94.66,18.93,0.37
2,1009,1,1.0,37.49,37.49,37.49,12.50,12.50,12.50,0.25,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.0,0.0,0.0,0,0.0,0.0,0,0,3,3.0,110.25,110.25,36.75,96.75,96.75,32.25,0.36
3,1272,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,1.0,29.26,29.26,29.26,14.74,14.74,14.74,0.28,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,78.46,78.46,39.23,39.54,39.54,19.77,0.28
4,1426,2,1.0,111.71,55.86,55.86,56.29,28.14,28.14,0.27,1,0.0,45.88,22.94,45.88,23.12,11.56,23.12,0.14,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,1.0,111.71,55.86,55.86,56.29,28.14,28.14,0.27


In [21]:
table2.shape

(257, 37)

In [20]:
May_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N = 0
for i,row in table2.iterrows():
    if N%100 == 0:
        print N
    N+=1
    #print row.Ilink
    key = getHigherPref(row)
    if type(key) is list:
        for x in key:
            May_Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        May_Users[key].append(row.Ilink)

0
100
200


In [23]:
len(May_Users[dept])

178

In [61]:
deptN = len(May_Users[dept])
not_deptN = len(May_Users['Dresses']) + len(May_Users['Knit Tops']) + len(May_Users['Woven Shirts'])
print not_deptN*1.0/deptN
print deptN*1.0/table2.shape[0]

0.455056179775
0.692607003891


## 3. Look at June-July

In [32]:
table3 = getTableRating(ilink_ex,df3)
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table3 = table3.loc[~(table3[cols]==0).all(axis=1)]
table3.reset_index(drop=True,inplace=True)

In [33]:
table3.shape

(254, 37)

In [34]:
June_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N = 0
for i,row in table3.iterrows():
    if N%100 == 0:
        print N
    N+=1
    #print row.Ilink
    key = getHigherPref(row)
    if type(key) is list:
        for x in key:
            June_Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        June_Users[key].append(row.Ilink)

0
100
200


In [35]:
len(June_Users[dept])

170

In [60]:
deptN = len(June_Users[dept])
not_deptN = len(June_Users['Dresses']) + len(June_Users['Knit Tops']) + len(June_Users['Woven Shirts'])
print not_deptN*1.0/deptN
print deptN*1.0/table3.shape[0]

0.529411764706
0.669291338583


## 4. Look at July-Aug

In [37]:
table4 = getTableRating(ilink_ex,df4)
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table4 = table4.loc[~(table4[cols]==0).all(axis=1)]
table4.reset_index(drop=True,inplace=True)

/Users/alexanderpan/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/alexanderpan/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [52]:
table4.shape

(145, 37)

In [53]:
July_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N = 0
for i,row in table4.iterrows():
    if N%100 == 0:
        print N
    N+=1
    #print row.Ilink
    key = getHigherPref(row)
    if type(key) is list:
        for x in key:
            July_Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        July_Users[key].append(row.Ilink)

0
100


In [54]:
len(July_Users[dept])

91

In [58]:
deptN = len(July_Users[dept])
not_deptN = len(July_Users['Dresses']) + len(July_Users['Knit Tops']) + len(July_Users['Woven Shirts'])
print not_deptN*1.0/deptN
print deptN*1.0/table4.shape[0]

0.615384615385
0.627586206897
